# Othello Move Predictor

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import random
import numpy as np
from pathlib import Path
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
from fastai.callbacks.mem import PeakMemMetric
from Othello_CNN import OthelloCNN
from torchsummary import summary

## Testing numpy i/o

In [ ]:
# Create matrices to concatenate
matrix_1 = np.zeros( shape=(8,8) )
matrix_2 = np.ones( shape=(8,8) )

# Stack depth-wise
matrix_3d = np.dstack( (matrix_1, matrix_2) )
matrix_3d.shape

# Save the numpy matrix
path = Path('/home/warren/Documents/5th_Year/AI')
name = path / 'test.npy'
np.save(name,matrix_3d)

In [ ]:
# Load numpy file and examine shape
mat = np.load(name)
mat.shape

In [ ]:
# Redefine open_image
def open_image(fn:PathOrStr, div:bool=True, convert_mode:str='RGB', cls:type=Image,
        after_open:Callable=None)->Image:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning) # EXIF warning from TiffPlugin
        x = PIL.Image.open(fn).convert(convert_mode)
    if after_open: x = after_open(x)
    x = pil2tensor(x,np.float32)
#     if div: x.div_(255)
    return cls(x)

fastai.vision.Image.open_image = open_image

In [ ]:
data.valid_ds[0]

In [2]:
# Batch size
bs = 256

# Define path to images
data_path = Path( '/users/ugrad/pretoriw/Documents/5th_Year/AI/numpy_boards_moves' )

# Get a list of names of all images in folder
# fnames = get_image_files( data_path )
fnames = os.listdir('/users/ugrad/pretoriw/Documents/5th_Year/AI/numpy_boards_moves')
split_idx = int(0.15 * len(fnames))

random.shuffle(fnames)
fnames_train = fnames[:split_idx]
fnames_val = fnames[split_idx:]

# Define function to retrieve label from filename
def get_label(file_path):
    file_name = file_path.name   # Get the file name (not path)
    label = file_name[:2]        # Get the first two chars of name

class NumpyDataset(Dataset):
    def __init__(self, data_path, fnames ):
        self.path = data_path
        self.names = fnames
        
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, idx):
        file_path = self.path/self.names[idx]
        
        img = np.load( file_path )
        img = np.float32(img)
        img = torch.Tensor( img )
        
        label = get_label( file_path )
        
        return img, label
    
train_dataset = NumpyDataset( data_path, fnames_train )
val_dataset = NumpyDataset(data_path, fnames_val)

train_dataloader = DataLoader( train_dataset, batch_size = bs)
val_dataloader = DataLoader( val_dataset, batch_size = bs)

data = ImageDataBunch(train_dataloader, val_dataloader)
data.c = 2

In [ ]:
# Batch size
bs = 256

# Define path to images
data_path = Path( '/users/ugrad/pretoriw/Documents/5th_Year/AI/numpy_boards_moves' )

# Get a list of names of all images in folder
fnames = get_image_files( data_path )

# Define function to retrieve label from filename
def get_labels(file_path):
    file_name = file_path.name   # Get the file name (not path)
    label = file_name[:2]        # Get the first two chars of name

# Load data into Databunch
data = ImageDataBunch.from_name_func( data_path, fnames, label_func=get_labels, bs=bs)

print(data.classes), data.c, data.show_batch( rows = 3 )

In [3]:
# Load model onto GPU..?
device = torch.device('cuda')
model = OthelloCNN().to(device)

# Create a learner
learn = cnn_learner( data, model, metrics = error_rate )

# Find the optimal learning rate
lr_find( learn )
learn.recorder.plot()

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not bool

In [ ]:
lr = 1e-3

learn.fit_one_cycle( 4, lr )

learn.save( 'stage1' )
learn.load( 'stage1' )